<a href="https://colab.research.google.com/github/Placeblanking/Internship/blob/Intership-projects/Tender.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
!pip install chardet
!pip install ipywidgets --quiet
!pip install pandas scikit-learn

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 20.9 MB/s eta 0:00:00


In [15]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_absolute_error
import ipywidgets as widgets
from IPython.display import display, clear_output

# ===== Load & Clean Data =====
CSV_PATH = "/content/SampleData1.csv"  # Change this path
df_raw = pd.read_csv(CSV_PATH)
df = df_raw.copy()
df.columns = df.columns.str.strip().str.lower()

# ===== Validation =====
required_cols = ['company name', 'product name', 'rate quoted', 'tabs', 'qty', 'region', 'units', 'zone']
missing = [c for c in required_cols if c not in df.columns]
if missing:
    raise ValueError(f"Missing required columns: {missing}")

# ===== Convert & Filter =====
df['tabs'] = pd.to_numeric(df['tabs'], errors='coerce')
df['qty'] = pd.to_numeric(df['qty'], errors='coerce')
df = df.dropna(subset=['rate quoted', 'tabs'])
df = df[df['tabs'] > 0]
df['rate_per_tab'] = df['rate quoted'] / df['tabs']
df = df.replace([np.inf, -np.inf], np.nan).dropna(subset=['rate_per_tab'])

# ===== Outlier Removal =====
low_q, high_q = df['rate_per_tab'].quantile([0.01, 0.99])
df = df[(df['rate_per_tab'] >= low_q) & (df['rate_per_tab'] <= high_q)]

# ===== Encode Categories =====
cat_cols = ['company name', 'product name', 'region', 'units', 'zone']
encoders = {}
for col in cat_cols:
    le = LabelEncoder()
    df[col] = le.fit_transform(df[col].astype(str))
    encoders[col] = le

company_names = encoders['company name'].classes_
product_names = encoders['product name'].classes_
zone_names = encoders['zone'].classes_

# ===== Model Training =====
features = ['company name', 'product name', 'qty', 'region', 'units', 'zone']
X = df[features]
y = df['rate_per_tab']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
model = RandomForestRegressor(n_estimators=300, random_state=42)
model.fit(X_train, y_train)

y_pred = model.predict(X_test)
mae = mean_absolute_error(y_test, y_pred)
print(f"Model MAE: {mae:.4f}")

# ===== Helper Functions =====
def competitor_rank_table(df_filtered, product_code, zone_filter):
    if zone_filter != "all":
        df_filtered = df_filtered[df_filtered['zone'] == zone_filter]
    df_filtered = df_filtered[df_filtered['product name'] == product_code]
    df_filtered = df_filtered.groupby('company name')['rate_per_tab'].median().reset_index()
    df_filtered['rank'] = df_filtered['rate_per_tab'].rank(method='first')
    df_filtered['company name'] = encoders['company name'].inverse_transform(df_filtered['company name'])
    return df_filtered.sort_values('rate_per_tab')

def suggest_bhl_rate_per_tab(rank_df):
    if len(rank_df) > 0:
        return rank_df.iloc[0]['rate_per_tab'] * 0.99  # Beat L1 by 1%
    return None

def clip_to_history(rate, df_hist):
    if len(df_hist) == 0:
        return rate
    min_hist = df_hist['rate_per_tab'].quantile(0.05)
    max_hist = df_hist['rate_per_tab'].quantile(0.95)
    return np.clip(rate, min_hist, max_hist)

# ===== UI Widgets =====
product_dropdown = widgets.Dropdown(
    options=[(name, i) for i, name in enumerate(product_names)],
    description='Product:',
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='40%')
)

zone_dropdown = widgets.Dropdown(
    options=[("All", "all")] + [(name, i) for i, name in enumerate(zone_names)],
    description='Zone:',
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='40%')
)

company_dropdown = widgets.Dropdown(
    options=[(name, i) for i, name in enumerate(company_names)],
    description='Company:',
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='40%')
)

mode_toggle = widgets.ToggleButtons(
    options=['Model Prediction', 'BHL Beat L1'],
    description='Mode:',
    style={'description_width': 'initial'}
)

beat_slider = widgets.FloatSlider(
    value=1.0, min=0.1, max=5.0, step=0.1,
    description='% Beat L1:',
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='40%')
)

output = widgets.Output()

# ===== Main Callback =====
def on_change(_change):
    with output:
        clear_output()

        prod_idx = product_dropdown.value
        zone_idx = zone_dropdown.value
        comp_idx = company_dropdown.value
        mode = mode_toggle.value
        beat_pct = beat_slider.value

        prod_name = product_names[prod_idx]
        comp_name = company_names[comp_idx]

        # Filter for competitor table
        rank_df = competitor_rank_table(df, prod_idx, zone_idx)
        print(f"Competitor Ranking for Product: {prod_name} | Zone: {zone_idx if zone_idx != 'all' else 'All'}")
        display(rank_df)

        # Prediction
        if mode == 'Model Prediction':
            test_input = pd.DataFrame([[
                comp_idx, prod_idx, df['qty'].median(),
                df['region'].mode()[0], df['units'].mode()[0],
                df['zone'].mode()[0] if zone_idx == 'all' else zone_idx
            ]], columns=features)
            pred_rate = model.predict(test_input)[0]
            hist_df = df[df['product name'] == prod_idx]
            pred_rate = clip_to_history(pred_rate, hist_df)
            print(f"Predicted Rate per Tab: {pred_rate:.4f}")

        elif mode == 'BHL Beat L1':
            suggested_rate = suggest_bhl_rate_per_tab(rank_df)
            if suggested_rate is not None:
                suggested_rate *= (1 - beat_pct/100)
                hist_df = df[df['product name'] == prod_idx]
                suggested_rate = clip_to_history(suggested_rate, hist_df)
                print(f"Suggested Rate per Tab to Beat L1: {suggested_rate:.4f}")
            else:
                print("No competitor data available.")

# ===== Event Listeners =====
product_dropdown.observe(on_change, names='value')
zone_dropdown.observe(on_change, names='value')
company_dropdown.observe(on_change, names='value')
mode_toggle.observe(on_change, names='value')
beat_slider.observe(on_change, names='value')

# ===== Display UI =====
display(
    widgets.HBox([product_dropdown, zone_dropdown]),
    widgets.HBox([company_dropdown]),
    widgets.HBox([mode_toggle, beat_slider]),
    output
)


Model MAE: 1.2146


Output()

In [17]:
!pip install lightgbm

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 36.7 MB/s eta 0:00:00


In [44]:
import pandas as pd
import numpy as np
import lightgbm as lgb
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.metrics import mean_absolute_error
from sklearn.preprocessing import OrdinalEncoder
import ipywidgets as widgets
from IPython.display import display, clear_output

# ===== Load & Clean Data =====
CSV_PATH = "/content/SampleData1.csv"
df = pd.read_csv(CSV_PATH)
df.columns = df.columns.str.strip().str.lower()

required_cols = ['company name', 'product name', 'rate quoted', 'tabs', 'qty', 'region', 'units', 'zone']
missing = [c for c in required_cols if c not in df.columns]
if missing:
    raise ValueError(f"Missing required columns: {missing}")

df['tabs'] = pd.to_numeric(df['tabs'], errors='coerce')
df['qty'] = pd.to_numeric(df['qty'], errors='coerce')
df = df.dropna(subset=['rate quoted', 'tabs'])
df = df[df['tabs'] > 0]

# ===== Feature Engineering =====
df['rate_per_tab'] = df['rate quoted'] / df['tabs']
df = df.replace([np.inf, -np.inf], np.nan).dropna(subset=['rate_per_tab'])

# Outlier removal (1–99 percentile)
low_q, high_q = df['rate_per_tab'].quantile([0.01, 0.99])
df = df[(df['rate_per_tab'] >= low_q) & (df['rate_per_tab'] <= high_q)]

# Log transform target for stability
df['log_rate_per_tab'] = np.log1p(df['rate_per_tab'])

# Encode categorical columns
cat_cols = ['company name', 'product name', 'region', 'units', 'zone']
encoder = OrdinalEncoder(handle_unknown="use_encoded_value", unknown_value=-1)
df[cat_cols] = encoder.fit_transform(df[cat_cols].astype(str))

company_names = encoder.categories_[0]
product_names = encoder.categories_[1]
zone_names = encoder.categories_[4]

# ===== Features & Target =====
features = ['company name', 'product name', 'qty', 'region', 'units', 'zone']
X = df[features]
y = df['log_rate_per_tab']

# ===== Train-Test Split =====
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# ===== LightGBM Model & Hyperparameter Tuning =====
lgb_model = lgb.LGBMRegressor(objective='regression', boosting_type='gbdt', random_state=42)

param_dist = {
    'num_leaves': [31, 50, 70, 100],
    'max_depth': [-1, 10, 20, 30],
    'learning_rate': [0.01, 0.05, 0.1],
    'n_estimators': [300, 500, 800],
    'min_child_samples': [10, 20, 30, 50]
}

search = RandomizedSearchCV(
    lgb_model,
    param_distributions=param_dist,
    n_iter=20,
    scoring='neg_mean_absolute_error',
    cv=3,
    random_state=42,
    verbose=0,
    n_jobs=-1
)

search.fit(X_train, y_train)
best_model = search.best_estimator_

# Evaluate
y_pred = best_model.predict(X_test)
mae = mean_absolute_error(np.expm1(y_test), np.expm1(y_pred))
print(f"Model MAE: {mae:.4f}")

# Retrain on full data
best_model.fit(X, y)

# ===== Helper Functions =====
def competitor_rank_table(df_filtered, product_code, zone_filter):
    if zone_filter != "all":
        df_filtered = df_filtered[df_filtered['zone'] == zone_filter]
    df_filtered = df_filtered[df_filtered['product name'] == product_code]
    df_filtered = df_filtered.groupby('company name')['rate_per_tab'].median().reset_index()
    df_filtered['rank'] = df_filtered['rate_per_tab'].rank(method='first')
    df_filtered['company name'] = company_names[df_filtered['company name'].astype(int)]
    return df_filtered.sort_values('rate_per_tab')

def suggest_bhl_rate_per_tab(rank_df):
    if len(rank_df) > 0:
        return rank_df.iloc[0]['rate_per_tab'] * 0.99
    return None

def clip_to_history(rate, df_hist):
    if len(df_hist) == 0:
        return rate
    min_hist = df_hist['rate_per_tab'].quantile(0.05)
    max_hist = df_hist['rate_per_tab'].quantile(0.95)
    return np.clip(rate, min_hist, max_hist)

# ===== UI Widgets =====
product_dropdown = widgets.Dropdown(
    options=[(name, i) for i, name in enumerate(product_names)],
    description='Product:',
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='40%')
)

zone_dropdown = widgets.Dropdown(
    options=[("All", "all")] + [(name, i) for i, name in enumerate(zone_names)],
    description='Zone:',
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='40%')
)

company_dropdown = widgets.Dropdown(
    options=[(name, i) for i, name in enumerate(company_names)],
    description='Company:',
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='40%')
)

mode_toggle = widgets.ToggleButtons(
    options=['Model Prediction', 'BHL Beat L1'],
    description='Mode:',
    style={'description_width': 'initial'}
)

beat_slider = widgets.FloatSlider(
    value=1.0, min=0.1, max=5.0, step=0.1,
    description='% Beat L1:',
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='40%')
)

output = widgets.Output()

# ===== Main Callback =====
def on_change(_change):
    with output:
        clear_output()
        prod_idx = product_dropdown.value
        zone_idx = zone_dropdown.value
        comp_idx = company_dropdown.value
        mode = mode_toggle.value
        beat_pct = beat_slider.value

        prod_name = product_names[prod_idx]
        comp_name = company_names[comp_idx]

        rank_df = competitor_rank_table(df, prod_idx, zone_idx)
        print(f"Competitor Ranking for Product: {prod_name} | Zone: {zone_idx if zone_idx != 'all' else 'All'}")
        display(rank_df)

        if mode == 'Model Prediction':
            test_input = pd.DataFrame([[comp_idx, prod_idx, df['qty'].median(),
                                        df['region'].mode()[0], df['units'].mode()[0],
                                        df['zone'].mode()[0] if zone_idx == 'all' else zone_idx]],
                                      columns=features)
            pred_log_rate = best_model.predict(test_input)[0]
            pred_rate = np.expm1(pred_log_rate)
            hist_df = df[df['product name'] == prod_idx]
            pred_rate = clip_to_history(pred_rate, hist_df)
            print(f"Predicted Rate per Tab: {pred_rate:.4f}")

        elif mode == 'BHL Beat L1':
            suggested_rate = suggest_bhl_rate_per_tab(rank_df)
            if suggested_rate is not None:
                suggested_rate *= (1 - beat_pct/100)
                hist_df = df[df['product name'] == prod_idx]
                suggested_rate = clip_to_history(suggested_rate, hist_df)
                print(f"Suggested Rate per Tab to Beat L1: {suggested_rate:.4f}")
            else:
                print("No competitor data available.")

# ===== Event Listeners =====
product_dropdown.observe(on_change, names='value')
zone_dropdown.observe(on_change, names='value')
mode_toggle.observe(on_change, names='value')
beat_slider.observe(on_change, names='value')

# ===== Display UI =====
display(
    widgets.HBox([product_dropdown, zone_dropdown]),
    widgets.HBox([mode_toggle, beat_slider]),
    output
)


Model MAE: 1.4062


Output()

In [47]:
pip install catboost

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.2/99.2 MB 11.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.3/47.3 kB 3.0 MB/s eta 0:00:00


In [49]:
import pandas as pd
import numpy as np
from catboost import CatBoostRegressor, Pool
from sklearn.metrics import mean_absolute_error
import ipywidgets as widgets
from IPython.display import display, clear_output

# ===== Load & Clean Data =====
CSV_PATH = "/content/SampleData1.csv"
df = pd.read_csv(CSV_PATH)
df.columns = df.columns.str.strip().str.lower()

required_cols = ['company name', 'product name', 'rate quoted', 'tabs', 'qty', 'region', 'units', 'zone', 'publish date']
missing = [c for c in required_cols if c not in df.columns]
if missing:
    raise ValueError(f"Missing required columns: {missing}")

df['tabs'] = pd.to_numeric(df['tabs'], errors='coerce')
df['qty'] = pd.to_numeric(df['qty'], errors='coerce')
df = df.dropna(subset=['rate quoted', 'tabs'])
df = df[df['tabs'] > 0]

df['rate_per_tab'] = df['rate quoted'] / df['tabs']
df = df.replace([np.inf, -np.inf], np.nan).dropna(subset=['rate_per_tab'])

# Outlier removal (1–99 percentile)
low_q, high_q = df['rate_per_tab'].quantile([0.01, 0.99])
df = df[(df['rate_per_tab'] >= low_q) & (df['rate_per_tab'] <= high_q)]

# ===== Feature Engineering =====
df['publish_date'] = pd.to_datetime(df['publish date'])
df['month'] = df['publish_date'].dt.month
df['year'] = df['publish_date'].dt.year

# Historical aggregated features
df['company_avg_rate'] = df.groupby('company name')['rate_per_tab'].transform('mean')
df['product_avg_rate'] = df.groupby('product name')['rate_per_tab'].transform('mean')
df['product_std_rate'] = df.groupby('product name')['rate_per_tab'].transform('std')
df['company_product_avg'] = df.groupby(['company name','product name'])['rate_per_tab'].transform('mean')

# Interaction features
df['zone_product'] = df['zone'].astype(str) + "_" + df['product name'].astype(str)
df['company_zone'] = df['company name'].astype(str) + "_" + df['zone'].astype(str)

# ===== Features & Target =====
cat_cols = ['company name','product name','region','units','zone','zone_product','company_zone']
num_cols = ['qty','company_avg_rate','product_avg_rate','product_std_rate','company_product_avg','month','year']
features = cat_cols + num_cols
target = 'rate_per_tab'

# ===== Train-Test Split (Time-based) =====
cutoff_date = df['publish_date'].max() - pd.Timedelta(days=90)  # last 3 months as test
train_df = df[df['publish_date'] < cutoff_date]
test_df = df[df['publish_date'] >= cutoff_date]

X_train = train_df[features]
y_train = train_df[target]
X_test = test_df[features]
y_test = test_df[target]

cat_indices = [X_train.columns.get_loc(c) for c in cat_cols]

# ===== CatBoost Model =====
model = CatBoostRegressor(
    iterations=1000,
    learning_rate=0.05,
    depth=6,
    eval_metric='MAE',
    random_seed=42,
    early_stopping_rounds=50,
    verbose=0
)

train_pool = Pool(X_train, y_train, cat_features=cat_indices)
test_pool = Pool(X_test, y_test, cat_features=cat_indices)

model.fit(train_pool, eval_set=test_pool)

# Evaluate
y_pred = model.predict(X_test)
mae = mean_absolute_error(y_test, y_pred)
print(f"CatBoost Model MAE: {mae:.4f}")

# Retrain on full data
full_pool = Pool(df[features], df[target], cat_features=cat_indices)
model.fit(full_pool, verbose=0)

# ===== Helper Functions =====
company_names = df['company name'].unique()
product_names = df['product name'].unique()
zone_names = df['zone'].unique()

def competitor_rank_table(df_filtered, product_code, zone_filter):
    if zone_filter != "all":
        df_filtered = df_filtered[df_filtered['zone'] == zone_filter]
    df_filtered = df_filtered[df_filtered['product name'] == product_code]
    df_filtered = df_filtered.groupby('company name')['rate_per_tab'].median().reset_index()
    df_filtered['rank'] = df_filtered['rate_per_tab'].rank(method='first')
    return df_filtered.sort_values('rate_per_tab')

def suggest_bhl_rate_per_tab(rank_df):
    if len(rank_df) > 0:
        return rank_df.iloc[0]['rate_per_tab'] * 0.99
    return None

def clip_to_history(rate, df_hist):
    if len(df_hist) == 0:
        return rate
    min_hist = df_hist['rate_per_tab'].quantile(0.05)
    max_hist = df_hist['rate_per_tab'].quantile(0.95)
    return np.clip(rate, min_hist, max_hist)

# ===== UI Widgets =====
product_dropdown = widgets.Dropdown(
    options=[(str(p), p) for p in product_names],
    description='Product:', style={'description_width':'initial'},
    layout=widgets.Layout(width='40%')
)
zone_dropdown = widgets.Dropdown(
    options=[('All','all')]+[(str(z), z) for z in zone_names],
    description='Zone:', style={'description_width':'initial'},
    layout=widgets.Layout(width='40%')
)
company_dropdown = widgets.Dropdown(
    options=[(str(c), c) for c in company_names],
    description='Company:', style={'description_width':'initial'},
    layout=widgets.Layout(width='40%')
)
qty_input = widgets.IntText(
    value=10,
    description='Qty:',
    style={'description_width':'initial'},
    layout=widgets.Layout(width='20%')
)
mode_toggle = widgets.ToggleButtons(
    options=['Model Prediction','BHL Beat L1'],
    description='Mode:', style={'description_width':'initial'}
)
beat_slider = widgets.FloatSlider(
    value=1.0, min=0.1, max=5.0, step=0.1,
    description='% Beat L1:', style={'description_width':'initial'},
    layout=widgets.Layout(width='40%')
)
output = widgets.Output()

# ===== Main Callback =====
def on_change(_):
    with output:
        clear_output()
        prod = product_dropdown.value
        zone = zone_dropdown.value
        comp = company_dropdown.value
        qty = qty_input.value
        mode = mode_toggle.value
        beat_pct = beat_slider.value

        rank_df = competitor_rank_table(df, prod, zone)
        print(f"Competitor Ranking for Product: {prod} | Zone: {zone}")
        display(rank_df)

        if mode == 'Model Prediction':
            # Build input
            input_df = pd.DataFrame([{
                'company name': comp,
                'product name': prod,
                'region': df['region'].mode()[0],
                'units': df['units'].mode()[0],
                'zone': zone if zone != 'all' else df['zone'].mode()[0],
                'zone_product': f"{zone if zone != 'all' else df['zone'].mode()[0]}_{prod}",
                'company_zone': f"{comp}_{zone if zone != 'all' else df['zone'].mode()[0]}",
                'qty': qty,
                'company_avg_rate': df[df['company name']==comp]['rate_per_tab'].mean(),
                'product_avg_rate': df[df['product name']==prod]['rate_per_tab'].mean(),
                'product_std_rate': df[df['product name']==prod]['rate_per_tab'].std(),
                'company_product_avg': df[(df['company name']==comp)&(df['product name']==prod)]['rate_per_tab'].mean(),
                'month': df['publish_date'].max().month,
                'year': df['publish_date'].max().year
            }])
            pred_rate = model.predict(input_df)[0]
            hist_df = df[df['product name']==prod]
            pred_rate = clip_to_history(pred_rate, hist_df)
            print(f"Predicted Rate per Tab: {pred_rate:.4f}")

        elif mode == 'BHL Beat L1':
            suggested_rate = suggest_bhl_rate_per_tab(rank_df)
            if suggested_rate is not None:
                suggested_rate *= (1 - beat_pct/100)
                hist_df = df[df['product name']==prod]
                suggested_rate = clip_to_history(suggested_rate, hist_df)
                print(f"Suggested Rate per Tab to Beat L1: {suggested_rate:.4f}")
            else:
                print("No competitor data available.")

# ===== Event Listeners =====
for w in [product_dropdown, zone_dropdown, company_dropdown, qty_input, mode_toggle, beat_slider]:
    w.observe(on_change, names='value')

# ===== Display UI =====
display(
    widgets.HBox([product_dropdown, zone_dropdown, company_dropdown, qty_input]),
    widgets.HBox([mode_toggle, beat_slider]),
    output
)


/tmp/ipython-input-1864196461.py:31: UserWarning: Parsing dates in %d-%m-%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  df['publish_date'] = pd.to_datetime(df['publish date'])


CatBoost Model MAE: 1.1979


Output()